In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../data/MachineLearningRating_v3.txt", delimiter="|", encoding="utf-8")
df.info()
df.describe(include='all')
df.isnull().sum()
df = pd.read_csv(
    "../data/MachineLearningRating_v3.txt",
    delimiter="|",
    encoding="utf-8",
    dtype={df.columns[32]: str, df.columns[37]: str},
    low_memory=False
)


In [ ]:
#EDA
# Convert TransactionMonth to datetime and extract period
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')
df['YearMonth'] = df['TransactionMonth'].dt.to_period('M')

# Create Loss Ratio column
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']


In [ ]:
#OVER ALL LOSS RATIO
overall_loss_ratio = df['TotalClaims'].sum() / df['TotalPremium'].sum()
print(f"Overall Loss Ratio: {overall_loss_ratio:.2f}")

In [ ]:
#Loss Ratio by Province, Gender, VehicleType
def group_loss_ratio(group_field):
    group = df.groupby(group_field)[['TotalClaims', 'TotalPremium']].sum()
    group['LossRatio'] = group['TotalClaims'] / group['TotalPremium']
    return group.sort_values('LossRatio', ascending=False)

loss_by_province = group_loss_ratio('Province')
loss_by_gender = group_loss_ratio('Gender')
loss_by_vehicle = group_loss_ratio('VehicleType')
print("Loss Ratio by Province:\n", loss_by_province)
print("Loss Ratio by Gender:\n", loss_by_gender)
print("Loss Ratio by Vehicle Type:\n", loss_by_vehicle)

In [ ]:
#Visualize loss by Province
sns.barplot(x=loss_by_province.index, y=loss_by_province['LossRatio'])
plt.xticks(rotation=45)
plt.title('Loss Ratio by Province')
plt.show()

In [ ]:
#outlier detection
plt.figure(figsize=(8, 4))
sns.boxplot(x=df['TotalClaims'])
plt.title("Boxplot of Total Claims")
plt.show()

plt.figure(figsize=(8, 4))
sns.histplot(df['CustomValueEstimate'].dropna(), kde=True)
plt.title("Distribution of Custom Value Estimate")
plt.show()

In [ ]:
#Time Based Trends
monthly_trends = df.groupby('YearMonth').agg({
    'TotalClaims': 'sum',
    'PolicyID': 'nunique'
})
monthly_trends['AvgClaimsPerPolicy'] = monthly_trends['TotalClaims'] / monthly_trends['PolicyID']

monthly_trends[['TotalClaims', 'AvgClaimsPerPolicy']].plot(figsize=(12, 6), title='Monthly Claim Trends')


In [ ]:
#Vehicle Make & Model Risk
vehicle_risk = df.groupby(['make', 'Model'])['TotalClaims'].mean().sort_values(ascending=False)

top10_models = vehicle_risk.head(10)
bottom10_models = vehicle_risk.tail(10)

top10_models.plot(kind='barh', title="Top 10 Vehicle Models by Avg Claim", figsize=(10,6))

plt.gca().invert_yaxis()

